In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 目的の場所（フォルダ・ディレクトリ）へ移動（各自の環境で適宜修正）
%cd "/content/drive/MyDrive/Colab Notebooks/JKJ1A/"
%ls


---
# 課題

## 課題１

実験課題1, 2をFGSMでなくPGDで行え．
- PGDでATしたモデルは，FGSMの時と同様，`model_cifar10_pgd0.010.pth`の様にして保存せよ．今後の便利のため，`alpha`, `n_iter`の値はファイル名に含めない．
- `advtrain_cifar10.py`を修正する形で行え．つまり以下のような感じで敵対的サンプルの生成方法を切り替えられるようにせよ（`set_args()`の中を修正することになる）．
```
!python advtrain_cifar10.py --attack_method 'fgsm'
!python advtrain_cifar10.py --attack_method 'pgd'
```
- `pgd`には`fgsm`でも用いた`eps`に加え二つ追加のパラメータがある．それぞれ，`alpha`, `n_iter`として，これも`set_args()`を修正することで，以下のようにできるようにせよ．
```
!python advtrain_cifar10.py --attack_method 'pgd' --eps 0.01, --alpha 0.002 --n_iter 7
```

補足：
PGDでのadversarial trainingは非常に遅い．またFGSM，PGDに関わらずadversarial trainingは通常よりも多めのepoch数での学習が必要になる．もし実行が難しいようなら，PGDの代わりにFastFGSMによるadversarial trainingをしても良い．FastFGSMはほとんどFGSMと同じだが，もともとの画像xに対して攻撃するのではなく，xを少しノイズで動かしたものにFGSMを行う．ノイズは`[-eps, eps]`の範囲の一様分布がよい．例えば`torch.rand_like`が利用できる．

```python 
x              # 元の画像
x = x + noise  # ノイズを加える
output = model(x)  #以降はFGSMと同じ
```

[Wong+'20](https://openreview.net/forum?id=BJx040EFvH): FastFGSMの元ネタ

## 課題２

これまで説明した敵対的サンプルの生成方法（例えばFGSM）では，「正しいラベルでの誤差が大きくなるように」画像を変化させ敵対的サンプルを生成した．これはUntargetedな方法（例えばUntargeted FGSM）と呼ばれる．この場合，敵対的サンプルがどの誤ったラベルになるかは事前にわからない．これに対してTargetedな方法は，「指定した誤ったラベルでの誤差が小さくなるように」画像を変化させる．Untargetedな方法が実装できていれば，ほんの少し修正すればTargetedな方法に修正できる．Targeted FGSM, Targeted PGDを実装し，`adversarial_attack.py`に追加せよ．Targetedな方法でいくつか敵対的サンプルを生成し，うまくいっていることを確認せよ（練習問題１の様な感じで）．

## 課題3A（最終レポート課題）
**この内容は最終レポートに含める課題の一つとする（3A or 3Bの選択）**

Adversarial attackは「人間の目には見えないほど小さな変化で分類器を騙す」というものである．この「人間の目には見えないほど」という制限を無くしたらどのようになるだろうか？

攻撃強度`epsilon`の制限を極端にゆるくした敵対的サンプルを生成し画像として表示し，その様子を観察せよ．`image`と`adv_image`の画像の間でどの様な変化が起きているだろうか？それぞれの画像に対しネットワークが出力した分類結果（クラス名）も並べて観察せよ（Day2 練習問題1の`cmp_images()`が参考になる）．通常の学習をしたモデル，adversarial trainingをしたモデルそれぞれについて同様に制限の緩い敵対的サンプルを生成し，その違いについても議論せよ．どのような変化が出るかはここでは言わないが，単に「めちゃくちゃ」になるのではなく興味深い変化が観察されるはず．課題2のTargetedな生成方法だと変化の傾向がわかりやすいかもしれない．

ここでの敵対的サンプル生成は，PGDで`epsilon=1.0`, `alpha=0.01`, `n_iter=100`あたりが良い．学習したモデルによるので，適宜修正が必要．極端にゆるくしてもだめで，少し調整する必要がある．TargetedとUntargetedでも異なる．

注意：モデルを学習する必要はなく，学習ずみのいくつかのモデルに対してPGD攻撃をすることになる．

---
追加課題（興味があればやってみよ）：
- 今回のAdversarial Attackは全て$L_{\infty}$ノルムに基づくものである．今回の課題は，$L_2$ノルムに基づくものだとより綺麗に結果が出る（ATも攻撃サンプル生成も$L_2$ノルムベースでやる）．`L1ノルム，L2ノルム`などで検索し$L_{p}$ノルムが何か把握した上で，調査しATの$L_2$ノルム版を実装し，上の課題をやってみよ．

## 課題3B（最終レポート課題）
**この内容は最終レポートに含める課題の一つとする（3A or 3Bの選択）**

新たなadversarial examples（のようなもの）を考案し，それに関して説明し，実験により検証せよ．実験事項として，
- どの程度攻撃に成功するかの検証，
- 生成したadversarial examplesの例示

は必ず行え．その上で考案のAE特有の点について実験・議論せよ．

網羅的に調査するのは難しいので，結果として真に新しい攻撃方法ではなかった，という場合でも減点等はしない．が，独創的・面白いものを期待する．

---
参考：特殊なAdversarial Examplesの例（これらを実装せよ，という意味ではない）
- [One-pixel attack] ただ一ピクセルだけ，ただしそのピクセルだけは自由に修正できる攻撃（[Su+,2017](https://arxiv.org/abs/1710.08864)）．
- [Universarial adversarial perturbations] 「画像に関係なくこの微小ノイズを足せばモデルを騙せる」というユニバーサルなノイズによる攻撃 ([Moosavi-Dezfooli+,2016](https://arxiv.org/abs/1610.08401))
- [Sparse fooling images] 「人間にとっては全く意味不明な画像」でモデルを騙す([Kumano+,2020](https://arxiv.org/abs/2012.03843))
